Script to test the python mzml loader 
==

In [5]:
import sys
sys.path.append('/Users/simon/git/lda/code/')

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load data using pymzml

In [13]:
data_dir = '/Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/'

In [81]:
from ms2lda_feature_extraction import LoadMZML
input_set = [data_dir + 'Beer_3_T10_POS.mzML']
peaklist = data_dir + 'Beer_3_full1.csv'
loader = LoadMZML(peaklist = peaklist,isolation_window = 0.5, mz_tol = 5, rt_tol = 10.0)
ms1,ms2,metadata = loader.load_spectra(input_set)

Loading spectra from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_T10_POS.mzML
Found 6680 ms2 spectra, and 191043 individual ms2 objects
Loaded 7863 ms1 peaks from /Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_full1.csv
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
Peaklist filtering results in 1582 documents


No. of MS1 peaks that can be matched to the peaklist

In [82]:
len(ms1)

1582

No. of MS2 peaks

In [83]:
len(ms2)

55785

### Filtering

*SR: not sure this step is necesary?*

Try to filter the pymzml results using the same filter criteria as in method 3's

In [88]:
# keep MS1 peaks with RT >3 and <21 mins
ms1 = filter(lambda x: x.rt>(3*60) and x.rt<(21*60), ms1)
ms2 = filter(lambda x: x[3].rt>(3*60) and x[3].rt<(21*60), ms2)

# keep MS2 peaks with intensity >5000
ms2 = filter(lambda x: x[2]>5000, ms2)

In [89]:
len(ms1), len(ms2)

(1459, 22665)

Now load method 3's results

In [19]:
import pandas as pd
from IPython.display import display, HTML

In [20]:
ms1_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms1.csv', 
                      index_col=0)
ms2_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms2.csv', 
                      index_col=0)

In [21]:
ms1_df.shape[0], ms2_df.shape[0]

(1289, 24287)

### Try to match the MS1 peaks from both results

In [84]:
from matching import MatchFeature, SimpleMatching

In [85]:
origin = 'pymzml'
left = []
for peak in ms1:
    f = MatchFeature(peak.mz, peak.rt, peak.intensity, origin)
    left.append(f)

In [86]:
origin = 'method_3'
right = []
for idx, row in ms1_df.iterrows():
    mz = row[['mz']].values[0]
    rt = row[['rt']].values[0]
    intensity = row[['intensity']].values[0]    
    f = MatchFeature(mz, rt, intensity, origin)
    right.append(f)

This does a matching (based on the m/z and RT only) for the MS1 peaks produced by the left (pyzml) and right (method-3) sides.

In [87]:
m = SimpleMatching()
matches = m.run(left, right, 2, 5)

Computing scores
Matching
1050 matched feature pairs
532 unmatched features from the first
239 unmatched features from the second


** These are out of date **

Justin said: the following big MS1 peaks should appear on both sides but they dont.

    found by R but not by Python:
    mz=267.126477, rt=633.804016, intensity=32300000.000000 origin=method_3 (3E7!)
    mz=278.123377, rt=542.280029, intensity=178000000.000000 origin=method_3 (1.5E8!)

    for the latter, Python found a similar mz at a different RT that could interfer? --> mz=278.123131, rt=595.984985, intensity=1340108.250000 origin=pymzml

    and the other way around:
    mz=401.172490, rt=205.735992, intensity=13729020.000000 origin=pymzml (1.2E7)
    mz=258.108396, rt=493.036987, intensity=12743071.000000 origin=pymzml (1.2E7)

In [66]:
for match in matches:
    items = list(match)
    if len(items) > 1: # a pair
        print items[0], '\n', items[1]
    else:
        print items[0] # single thing
    print

mz=86.096347, rt=276.096008, intensity=1282799.125000 origin=pymzml 
mz=86.096347, rt=276.096008, intensity=1282799.125000 origin=method_3

mz=99.044050, rt=587.219971, intensity=1300673.500000 origin=method_3 
mz=99.044050, rt=587.219971, intensity=1300673.500000 origin=pymzml

mz=157.080102, rt=622.317017, intensity=7317130.000000 origin=method_3 
mz=157.080102, rt=622.317017, intensity=7317130.000000 origin=pymzml

mz=210.169968, rt=278.946014, intensity=6319341.000000 origin=method_3 
mz=210.169968, rt=278.946014, intensity=6319341.000000 origin=pymzml

mz=90.054968, rt=628.815979, intensity=6755740.000000 origin=pymzml 
mz=90.054968, rt=628.815979, intensity=6755740.000000 origin=method_3

mz=84.080727, rt=567.288025, intensity=978819.375000 origin=pymzml 
mz=84.080727, rt=567.288025, intensity=978819.375000 origin=method_3

mz=249.192081, rt=526.107971, intensity=1228983.250000 origin=pymzml 
mz=249.192081, rt=526.107971, intensity=1228983.250000 origin=method_3

mz=348.188735, r